## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
## determine which models are not already fit ##
import models_to_be_fit
num_of_p_values = 1
num_of_prediction_months = len(prediction_months)
churn_risk_prediction_exports = 'data/'+date_dir+'/exports/churn_risk_prediction/'+model_type+'/'
coefficients_and_pvalues_exports = 'data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'
(already_fit, yet_to_be_fit) = models_to_be_fit.main(model_numbers=model_numbers,\
                                                     model_type=model_type,\
                                                     date_dir=date_dir,\
                                                     churn_risk_prediction_exports=churn_risk_prediction_exports,\
                                                     coefficients_and_pvalues_exports=coefficients_and_pvalues_exports,\
                                                     data_set_name=data_set_name,\
                                                     num_of_p_values=num_of_p_values,\
                                                     num_of_prediction_months=num_of_prediction_months)

# for model_number in properly_used_inquiries_models['model_number'].unique():
for model_number in yet_to_be_fit:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)
    
    churn_risk_calculation.calculate_churn_risk(date_of_analysis = date_of_analysis, df_timeline = df_timeline, \
                                            vars_ = list(ctv.summary.index), data_all_spots = data_all_spots, ctv = ctv, \
                                            model_type = model_type, model_number = model_number,\
                                            with_add_vars = False, p_limit = p_limit,\
                                            event_date_type=event_date_full_name, spots_set = spots_set, with_wo_CB=with_wo_CB)
    
    for prediction_month in prediction_months:
        testing_results = model_testing.\
        get_testing_results(date_of_analysis=date_of_analysis,\
                            df_timeline=df_timeline.copy(), \
                            base_df = base_df.copy(),\
                            data_all_spots=data_all_spots.copy(), \
                            hs_list_path=hs_list_path,\
                            model_type=model_type,\
                            model_number=model_number,\
                            with_add_vars=False, p_limit=p_limit, event_date_type=event_date_full_name, \
                            prediction_month=prediction_month, ctv=ctv, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, penalizer=penalizer, test_with_req_canc_before=True)

        model_testing.save_results(date_of_analysis=date_of_analysis,\
                               df=testing_results[0], \
                               results_name='testing_results', \
                               model_type=model_type,\
                               model_number=model_number,\
                               spots_set=spots_set,\
                               with_wo_CB=with_wo_CB,\
                               event_date_type=event_date_full_name,\
                               with_add_vars=False, p_limit=p_limit, prediction_month=prediction_month)
        display(Markdown("## Model "+ str(model_number) + ": testing results"))
        display(testing_results[0].set_index('index'))

# Model 38: properly_used_private_parties_inquiries

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 80406 periods, 5936 subjects, 1004 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 5936
 number of periods = 80406
  number of events = 1004
partial log-likelihood = -7145.09
  time fit was run = 2023-02-04 08:07:31 UTC

---
                                                                                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                                             
Annual Commitment with 2 semi-annual payments                                  -0.37       0.69       0.24            -0.83             0.10                 0.43                 1.11
Annual Prepay                                                                  -0.15       0.86       0.09            -0.33             0.03                 0.72                 1.03
Fb.page.likes.analysis.avg.log2                                                -0.03       0.97       0.01            -0.05            -0.01                 0.95                 0.99
Monthly                                                                         0.49       1.63       0.13             0.23             0.75                 1.26                 2.11
New.email.subscriber.signups.last.month.div10                                  -0.03       0.97       0.04            -0.10             0.04                 0.91                 1.04
New.email.subscribers.allinclusive.last.month.div10                            -0.03       0.97       0.02            -0.07             0.00                 0.93                 1.00
Opted.out.of.facebook                                                          -0.18       0.84       0.16            -0.49             0.13                 0.61                 1.14
SpotHopper_site_not_live_yet                                                    0.41       1.51       0.13             0.15             0.68                 1.16                 1.97
Website.Views.last.month.total.log2                                            -0.19       0.82       0.03            -0.24            -0.14                 0.78                 0.87
had_added_events_manually_edited_events_before_and_didnt_last_4_months          0.18       1.20       0.12            -0.05             0.41                 0.95                 1.51
had_added_events_manually_edited_events_last_4_months                           0.06       1.07       0.13            -0.19             0.32                 0.83                 1.37
had_added_food_edited_food_before_and_didnt_last_4_months                       0.13       1.14       0.10            -0.07             0.32                 0.94                 1.38
had_added_food_edited_food_last_4_months                                       -0.07       0.93       0.11            -0.29             0.14                 0.75                 1.15
had_added_specials_edited_specials_before_and_didnt_last_4_months              -0.04       0.96       0.09            -0.22             0.14                 0.80                 1.15
had_added_specials_edited_specials_last_4_months                                0.07       1.07       0.12            -0.17             0.31                 0.84                 1.36
had_bday_club_downloads_before_and_didnt_last_2_months                          0.36       1.43       0.35            -0.33             1.05                 0.72                 2.85
had_bday_club_downloads_last_2_months                                          -0.04       0.96       0.36            -0.75             0.67                 0.47                 1.95
had_catering_submissions_before_and_didnt_last_4_months                         0.11       1.11       0.19            -0.27             0.48                 0.76                 1.62
had_catering_submissions_last_4_m

## without categorical variables with |coeff| < 0.2 and p value > 0.2

## without numerical variables with |coeff| < 0.01 and p value > 0.2

...

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,3763,3701
requested_cancellation,36,17
prediction_month,2022-11-01,2022-11-01
pearsons_corr_coef_and_p_value,"(0.042, 0.011)","(0.025, 0.122)"
spearmans_corr_coef_and_p_value,"(0.056, 0.001)","(0.053, 0.001)"
log_loss,0.059056,0.029817
concordance,66.559,72.731
discordance,33.441,27.269
requested_cancellation_in_the_first_20_%_of_all_spots,12,7


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,4053,4010
requested_cancellation,39,18
prediction_month,2022-12-01,2022-12-01
pearsons_corr_coef_and_p_value,"(0.032, 0.043)","(0.043, 0.006)"
spearmans_corr_coef_and_p_value,"(0.062, 0.0)","(0.074, 0.0)"
log_loss,0.058286,0.026557
concordance,68.339,81.971
discordance,31.661,18.029
requested_cancellation_in_the_first_20_%_of_all_spots,19,11


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,4327,4305
requested_cancellation,30,8
prediction_month,2023-01-01,2023-01-01
pearsons_corr_coef_and_p_value,"(0.072, 0.0)","(0.057, 0.0)"
spearmans_corr_coef_and_p_value,"(0.088, 0.0)","(0.066, 0.0)"
log_loss,0.038138,0.012158
concordance,80.721,94.17
discordance,19.279,5.83
requested_cancellation_in_the_first_20_%_of_all_spots,21,8
